# Import of libraries

In [ ]:
import numpy as np
from matplotlib import pyplot
from struct import*
import pandas as pd
import math
import os
from numpy import nan
from matplotlib import pyplot as plt

# Reading file, shaping 3D matrix and plot of CT/PET.

In [ ]:
# Function to read info_text file.
def read_info_txt(file):
    with open(file, mode='r') as file: #Reading PET or CT file
        fileContent = file.read()
        SplitFile = fileContent.split()
        array_info_txt = []
        for i in range(len(SplitFile)):
            array_info_txt.append(float(SplitFile[i]))
    return array_info_txt


# Function to read data, and create a 3D matrix
def read_CT_PET(PatientNumber,Image='CT'):
    # Add up strings to form path to extract image
    Part_readfile = '/volumes/LaCie/Sagal/HN_ANON_09102017/Patient_Folders/'
    strek ='/'
    if Image == 'CT':
        Imaging = 'CT'
    else:
        Imaging = 'PT'
    file = Part_readfile+PatientNumber+strek+Imaging
    with open(file, mode='rb') as file: #Reading PET or CT file
        fileContent = file.read()
    Unpack = unpack("H" * ((len(fileContent)//2)), fileContent) # Converts data stored as bytes to an array.
    # Read InfoTextFile
    remaining = '_info.txt'
    InfoTextFile = Part_readfile + PatientNumber + strek + PatientNumber + remaining
    ydim,xdim,zdim,SUVMax,CTMax = read_info_txt(InfoTextFile)
    Matrix3D = []
    array_data = np.array(Unpack)
    for i in range(int(zdim)):
        Image_array = (array_data[i*int(xdim)*int(ydim):(i+1)*int(xdim)*int(ydim)]) # Creates array for each 2D image
        ReshapeImage = np.reshape(Image_array,[int(xdim),int(ydim)]) # Reshapes image to 2D
        if Image == 'CT':
            CT_array = ReshapeImage*CTMax/(2**16-1)
            Matrix3D.append(CT_array)
        else:
            PET_array = ReshapeImage*SUVMax/(2**16-1)
            Matrix3D.append(PET_array)
        Matrix3D_array = np.array(Matrix3D) # Create an array
        Matrix3D_new = np.squeeze(Matrix3D_array) # Get 3D matrix 
    return Matrix3D_new

Image_Matrix = read_CT_PET('P005','PT')

def plot(Layer):
    Layer_Image = Image_Matrix[Layer]
    plot = pyplot.imshow(Layer_Image)# Display 2D image
    return plot    

In [ ]:
def plot(Layer):
    Layer_Image = Image_Matrix[Layer]
    plot = pyplot.imshow(Layer_Image)# Display 2D image
    return plot 
plot(186)

# Read of raw_text, and extraction of z, y and x coordinate values.


In [ ]:
# Finding Unicode Error place. We have used a 'iso-8859-15' encoding. This seems to fix the error somewhat.
# This function reads where GTV starts from. 
def Search_GTV(file):
    enc = 'iso-8859-15'
    f = open(file,encoding=enc)
    lines =f.readlines()
    LineNumber = []
    Header = []
    for line_number, line in enumerate(lines):
        if 'GTV' in line:
            LineNumber.append(line_number)
            Header.append(line)
    return LineNumber,Header

# This is just a function that reads the file.
def readline(file,k):
    enc = 'iso-8859-15'
    with open(file,encoding=enc, mode='r') as file:
        for i in range(int(k)):
            fileContent = file.readline()
        filecontent = file.read()
        return filecontent

# This creates an array
def array_creator(SplitFile):
    array_info_txt = []
    for i in range(len(SplitFile)):
        try:
            array_info_txt.append(float(SplitFile[i]))
        except ValueError:
            break
    return array_info_txt

# This gets out whole array

def array(file):
    Start_point,GTV_Names = Search_GTV(file)
    filecontent = readline(file,Start_point[0]+1)
    SplitFile = filecontent.split()
    count = SplitFile.count('GTV')
    Array_Whole = []
    Position = []
    i = 0
    while i < count+1:
        if len(SplitFile) == 0:
            break
        else:
            Array = array_creator(SplitFile)
            position = len(Array)/3 + 2
            Position.append(position)
            Array_Whole.append(Array)
            filecontent = readline(file,Start_point[0]+1+np.sum(Position))
            SplitFile = filecontent.split()
        i += 1
    return Array_Whole



def Func_Concat_Header_Array_Index_PT_CT(file,Index_type='Index'):
    LineNumber,Header = Search_GTV(file)
    Array_Whole = array(file)
    Array_Index_Whole = []
    if Index_type == 'Index':
        for i in Array_Whole:
            Array = i[0::3]
            Array_Index_Whole.append(Array)
    elif Index_type == 'PT':
        for i in Array_Whole:
            Array = i[1::3]
            Array_Index_Whole.append(Array)
    elif Index_type == 'CT':
        for i in Array_Whole:
            Array = i[2::3]
            Array_Index_Whole.append(Array)
    Concat_Header_Array = []
    for j, i in enumerate(Array_Index_Whole):
        Concat_Header_Array.append([Header[j],i])
    return Concat_Header_Array

# Function to convert Index Values to Pixel Numbers
# Try to find voxel on which doctor has drawn lines.

def convert_index_to_pixel(PatientNumber):
    z_index = []
    y_index = []
    x_index = []
    Part_readfile = '/volumes/LaCie/Sagal/HN_ANON_09102017/Patient_Folders/'
    strek ='/'
    file_text = '_info.txt'
    file_raw = '_raw_data.txt'
    InfoTextFile = Part_readfile + PatientNumber + strek + PatientNumber + file_text
    RawTextFile  = Part_readfile + PatientNumber + strek + PatientNumber + file_raw
    Index_data = Func_Concat_Header_Array_Index_PT_CT(RawTextFile,Index_type='Index')
    names = []
    for i in Index_data:
        for index in i:
            if 'GTV' in index:
                names.append(index)
                continue
            else:
                ydim,xdim,zdim,SUVMax,CTMax = read_info_txt(InfoTextFile)
                z = np.floor(np.array(index)/(xdim*ydim))
                y = np.floor((np.array(index)-z*xdim*ydim)/(ydim))
                x = np.array(index) - z*xdim*ydim-y*ydim
            z_index.append(z)
            y_index.append(y)
            x_index.append(x)
    return z_index,y_index,x_index,names,SUVMax
# Husk du har lagt til SUVMax. Kan hende du opplever utfordringer når du kjører gammel kode.

# Create a function that reads out the correct tumor. We are intrested in GTV PET.

In [ ]:
# Reading in GTV names can be absolutely improved

In [ ]:
# Reading in GTV names from excel file

# Code reading in GTV names together with Patient Number.
df = pd.read_excel('GTVNames.xlsx',skiprows=14) # Cod reading in GTV names that shall be used. 
GTV = df.iloc[:,2] # Chosing correct column
GTVNames = GTV.to_numpy() # Convert to Numpy array

#Function reading out correct Tumor.
def CorrectTumorChoice(PatientNumber):

    #array_names = ['GTV prim tum CT (70 Gy) ','GTV/CT prim tum ( 70 Gy)','GTV prim tum CT (70 Gy) ','GTV CT (70 Gy)','GTVtumor/pb ','GTVtumor(70 gy)/henma ','GTVtumor/AN ','GTV/JFE (70 Gy) ','GTVtumor(70gy)/henma','GTV_PET_TUMOR ','GTV_Tumor_PET ','GTVtumor','GTV tumor/brt ','GTVvetonsille/henma ','GTV tumor 70 Gy- uxjolp ','GTV tumor (70 Gy) - uxjolp','GTV_PET_TUMOR (70 Gy) ','GTV prim tum PET/CT (70 Gy) ','GTV_tumor_PET_TVB ','GTV_TUMOR_PET_TVB ','GTV_PET_TUMOR_TVB ','GTV_PET_tumor_TVB ','GTV_PET_TVB ','GTV_PET_tvb ','GTV PET (70 Gy) ','GTV PET JFE (70 Gy) ','GTV primtum PET (70 Gy) ','GTV tumor PET CT','GTV_PET_tumor ','GTV_tumor_PET_tvb ','GTV_tumor_PET_AL','GTV_LNSIN_PET_TVB ','GTV_tumor_PET (70 Gy) ','GTV_tumor_PET ','GTV prim tum PET/CT,GTV prim tum PET/CT (70 Gy)','GTV PET prim tum (70 Gy)','GTV prim tum PET (70 Gy)','GTV prim tum PET','GTV prim tum PET ()70 Gy)', 'GTV_PET_TUMOR (70 Gy)','GTV_PRIMTUM_PET (70 Gy)','GTV CT','GTV PET (70 Gy)' ,'GTV prim tum (70 Gy)','GTV prim tum PET','GTV prim tum MR','GTV prim tum CT (70 Gy)' ,'GTV prim tum CT (70 Gy)','GTV tumor 70GY/MN','GTV_TUMOR_PET hs_TVB','GTV_tumor_PET_tvb','GTV primtum PET JFE (70 Gy)','GTV_TUMOR_PET_TVB','GTV_PET_tumor_TVB','GTV_TUMOR?_PET_TVB','GTV_TUMOR_PET_TVB 70 Gy' ,'GTV_PET_tumor_TVB','GTV_PET_TUMOR_TVB','GTV_tumor_PET_TVB' ,'GTV_tumor_PET_TVB (70 Gy)','GTV-PET l.gl ve 1/brt' ,'GTV-PET lgl ve2/brt','GTV tonsille (70 Gy)/pbk','GTV prim tum (68 Gy)','GTV_TUMOR_PET ','GTV prim tum','GTV_LNdex_PET','GTV_origo?_PET']
    array_names = GTVNames
    PatientWithTumor=[]
    if PatientNumber == 'P118':
        pass
    else:
        Z,Y,X,names,SUVMax = convert_index_to_pixel(PatientNumber)
    KO = []
    Names = []
    results = [] # Will contain array name, and Array number later.
    for i in names:
        string = i.strip('\n')
        Names.append(string)
    for i in range(len(Names)):
        Namy = Names[i]
        for k in array_names:
            if Namy == k:
                KO.append(Namy)
                KO.append(i)
                PatientWithTumor.append(PatientNumber)
                return Namy,i,PatientWithTumor
    empty_array = np.array(KO) # Brude denne snipp biten til å legge igjen de jeg ikke finner Tumor for. 
    if empty_array.size == 0:
        pass
        return print('No Tumor Found'),print(Names),print(PatientNumber)



# Function that creates patient list
def PatientList(Number):
    Patient_List = []
    PossibleAvailable = []
    for i in range(1,Number+1):
        PossibleAvailable.append(i)
        Standard = 'P'
        OneZero = '0'
        TwoZer0 = '00'
        digits = int(math.log10(i))+1 # Use this to code number of digits of 'i'
        if digits == 1:
            PatientNum = Standard + TwoZer0 + str(i)
        elif digits == 2:
            PatientNum = Standard + OneZero + str(i)
        else:
            PatientNum = Standard + str(i)
        Patient_List.append(PatientNum)
    return Patient_List

# In number you incert number of patients you want to check out
def Check_Funtion(Number):
    #Create a list with PET values for patients:-
    NumberAvailable = []
    UniCodeDecodeErrorAntall = []
    AbsentFilesNumber = []
    for i in (PatientList(Number)):
        try:
            GTVName,GTVPositionNumber,PatientWithTumor = CorrectTumorChoice(i)
            print(i,GTVName,GTVPositionNumber)
            if PatientWithTumor == None:
                pass
            else:
                NumberAvailable.append(PatientWithTumor)
        except FileNotFoundError:
            print(i,'This file is absent')
            AbsentFilesNumber.append(i)
        except TypeError:
            pass
        except UnboundLocalError:
            pass
    print('Number Available are',len(NumberAvailable))
    print('The Possible Available are',Number)
    print('Absent files are',len(AbsentFilesNumber))
    return NumberAvailable,AbsentFilesNumber
# Pasient nr 180 intressant. Den har flere prim tumor.
# Pasient nr 160 er jeg usikker på.

PatientWithTumors,AbsentFiles=(Check_Funtion(254))



# Creates a 3D matrix that icludes tumor voxel. 

In [ ]:
# This creates a 3D matrix that contains whole tumor. Also coordinates that lineates around the tumor is included. 
# In the code we use function CorrectTumorChoice. This choses for us the correct tumor.
def create_dummy_matrix(PatientNumber):
    pixel = convert_index_to_pixel(PatientNumber)
    z,y,x,names,suvmax = pixel
    Part_readfile = '/volumes/LaCie/Sagal/HN_ANON_09102017/Patient_Folders/'
    strek ='/'
    file_text = '_info.txt'
    file_raw = '_raw_data.txt'
    InfoTextFile = Part_readfile + PatientNumber + strek + PatientNumber + file_text
    ydim,xdim,zdim,SUVMax,CTMax = read_info_txt(InfoTextFile)
    z,y,x,name,suvmax = pixel
    try:
        ArrayName,ArrayNum,PatientWithTumor = CorrectTumorChoice(PatientNumber)
        z = (z[ArrayNum])# RART! Hvorfor ikke bare z?
        y = (y[ArrayNum]) # Rart! Hvorfor ikke bare y?
        x = (x[ArrayNum]) # RART!
        Dummy = np.zeros([int(zdim),int(xdim),int(ydim)])
        X_axis = []
        Y_axis = []
        Z_axis = []
        for i in range(len(z)):
            x1 = int(x[i])
            y1 = int(y[i])
            z1 = int(z[i])
            X_axis.append(x1)
            Y_axis.append(y1)
            Z_axis.append(z1)
            Dummy[z1,x1,y1] = 1000
        return Dummy,X_axis,Y_axis,Z_axis
    except TypeError:
        pass

# Function below visualizes in 2D
def PLOT_DUMMY(PATIENTnuMB):
    Dummy,X,Y,Z = create_dummy_matrix(PATIENTnuMB)
    y3 = np.array(Y)
    x3 = np.array(X)
    z3 = np.array(Z)
    Center_x =(np.amax(X)+np.amin(X))/2
    Center_y =(np.amax(Y)+np.amin(Y))/2
    Center_z =(np.amax(Z)+np.amin(Z))/2
    print(Center_x,Center_y,Center_z)
    fig = pyplot.figure()
    pyplot.plot(x3,y3,'*')
    pyplot.show()
    # Function below visualizes tumor in 3D
    from itertools import product, combinations
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    pyplot.rcParams["figure.figsize"] = [7.00, 3.50]
    pyplot.rcParams["figure.autolayout"] = True
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x3, y3, z3, c=z3, alpha=1)
    pyplot.show()
'''    Image_Matrix = read_CT_PET(PATIENTnuMB,'PT')
    Layer_Image = Image_Matrix[int(Center_z)]
    fig = plt.figure()
    pyplot.imshow(Layer_Image)'''# Display 2D image



# Calculate various PET parameters for ALL patients

## Code to collect SUV.

In [ ]:
# Function extracting SUVVALUES 
# From it we also obtain the Tumor volume.

def SUVExtractor(PatientNumber):
    PatientWithTumor = []  # Here I store patient with a primary tumor. 
    Part_readfile = '/volumes/LaCie/Sagal/HN_ANON_09102017/Patient_Folders/'
    strek ='/'
    file_raw = '_raw_data.txt'
    RawTextFile  = Part_readfile + PatientNumber + strek + PatientNumber + file_raw # Read in file
    Index_data = Func_Concat_Header_Array_Index_PT_CT(RawTextFile,Index_type='PT') # Extract index
    PET_VALUES = []
    for i in Index_data:
        for j in i:
            if j == 'Hi':
                pass
            else:
                PET_VALUES.append(j)
    # To chose correct array number FROM PET_VALUES, ARRAY NUM IS USED.
    try:
        ArrayName,ArrayNum,PatientWithTumor = CorrectTumorChoice(PatientNumber)     
        # Need to define a so that we chose the correct array
        a = 2*ArrayNum +1
        Pet_Values0 = PET_VALUES[a]
        PatientWithTumor.append(PatientNumber)
        return Pet_Values0
    except ValueError:
        pass

# Lese ut tumor verdi.
def SUV_samler(PatientName):
    SUVSAMLET = []
    for i in (PatientName):
        if i == 'P118': # Jumping over because it causes code to crash
            pass
        else:
            b = (SUVExtractor(i))
            a = np.array(b)
            SUVSAMLET.append(a)
    return SUVSAMLET

## Code to calculate PET parameters

In [ ]:
# Extract SUVMEAN from SUV-values obtained from files. 
def SUVMEAN(SUV):
    SUVMEANLIST = []
    for i in SUV:
        a = np.mean(i)
        SUVMEANLIST.append(a)
    return SUVMEANLIST

# Extract SUVMAX from SUV-values obtained from files. 

def SUVMAXMINE(SUV):
    SUVMAXLIST = []
    for i in SUV:
        a = np.amax(i)
        SUVMAXLIST.append(a)
    return SUVMAXLIST

#Tumor volume from the length of the PET-values for each pasient. 
def TumorVolumeFunction(SUV):
    TumorVolume = []
    PatientNumber = []
    for i in SUV:
        a = len(i)/1000
        TumorVolume.append(a)
    return TumorVolume


def MTV(SUV):
    MTV = []
    SuvMaximum = SUVMAXMINE(SUV) # Maximum SUV for patients are stored here. 
    SUVPixelValue = [] # SUV-values greater than threshold are collected here. 
    SUVCOLLECTER = []
    for k, i in enumerate (SUV):
        SuvMaX  = SuvMaximum[k]
        Threshold = SuvMaX*0.41
        if Threshold < 2.5:
            Threshold = 2.5
        for j in  i:
            if j > Threshold:
                SUVPixelValue.append(j)
        SUVCOLLECTER.append(SUVPixelValue)
        SUVPixelValue = []

    for i in SUVCOLLECTER:
        if i == []:
            pass
        else:
            a = len(i)/1000
            MTV.append(a)
    return MTV


def TLG(SUV):
    SuvMean=SUVMEAN(SUV)
    TLG = []
    for i,j in enumerate (MTV(SUV)):
        tlg = j*SuvMean[i]
        TLG.append(tlg)
    return TLG

def center_SUV(SUV,PatientGroup):
    Index = find_Center_Index(PatientGroup)
    Centerr_SUV=[]
    for i,j in enumerate (SUV):
        centerr=Index[i]
        Centerr=SUV[centerr]
        Centerr_SUV.append(Center)
        return Centerr_SUV

## Read in all patients and HPV positive and negative patients

In [ ]:
# Function that stores all patient groups.
def allpatients():
    AllPatients = []
    for i in PatientWithTumors:
        for j in i:
            AllPatients.append(j)
    return AllPatients

# Function used to read in HPV file.
def ReadHPVSituation():
    df = pd.read_csv('hpv_group_all.csv',skiprows=0)
    a = df.iloc[:,1]
    HPV_Situation = a.to_numpy()
    b = df.iloc[:,0]
    NumberOfPatient=b.to_numpy()
    return HPV_Situation,NumberOfPatient

# Read out patientnumber from file
def HPVPatientNumber():
    HPV_Situation,NumberOfPatient = ReadHPVSituation()
    # Grupperer pasienter i HPV postive = 1, og negative = 0. 
    PatientnumbOne = []
    PatientnumbZero = []
    HPVSITUATION = []

    for i,j in enumerate(HPV_Situation):
        if j == 0:
            PatientnumbZero.append(NumberOfPatient[i])
        elif j == 1:
            HPVSITUATION.append(j)
            PatientnumbOne.append(NumberOfPatient[i])
    return PatientnumbZero,PatientnumbOne

PatientnumbZero,PatientnumbOne = HPVPatientNumber()

# Code that Adds P and necessary zeros before patient id
def ExtractionPationNumbersHPV(HPVPatientsRelation):
    List = []
    for i in HPVPatientsRelation:
        Standard = 'P'
        OneZero = '0'
        TwoZer0 = '00'
        a = int(i)
        digits = int(math.log10(a))+1 # Use this to code number of digits of 'i'
        if digits == 1:
            PatientNum = Standard + TwoZer0 + str(a)
        elif digits == 2:
            PatientNum = Standard + OneZero + str(a)
        else:
            PatientNum = Standard + str(a)
        List.append(PatientNum)

    return (List)

# Function removing patients that do not have a GTV
def FinalPatientNumberHPV(PatientnumbZero):
    PatientNumberHPV0list = ExtractionPationNumbersHPV(PatientnumbZero)
    PatientNumberList=[]
    PatientWithTumor = allpatients()
    for i in PatientWithTumor:
        for k in PatientNumberHPV0list:
            if k == i:
                PatientNumberList.append(i)
    return PatientNumberList


#PatientNumber for HPV negative pasients are saved here.
PatientNumbeRHPVZero = FinalPatientNumberHPV(PatientnumbZero)
#PatientNumber for HPV positive pasients are saved here.
PatientNumbeRHPVOne = FinalPatientNumberHPV(PatientnumbOne)
#All pasients are saved here
AllPasientNumbers = allpatients()

## Extract outcome, and also corresponding patient numbers.

In [ ]:
#Function that reads inn files:
# ArrayNumber if pasient is not sick
Sykdomsfri = 90
#ArrayNumber if pasient survies
Survival = 79
#Alder
Ecog = 32
#MTV
MTV = 59
def ReadInFiles(ArrayNumber):
    #Read in Data
    df = pd.read_excel('pasinfo.xls',skiprows=2)
    # Reading in Patient ID
    ab = df.iloc[:,0]
    # Read in over all survival.
    a = df.iloc[:,ArrayNumber]
    # Remove Nan from list
    #Pasient = ab[:254]
    #PasientId = ab[np.logical_not(np.isnan(ab))]
    PasientId = ab[:254]
    # Remove Nan from list
    OverAllSurvival = a[:254]
    #OverAllSurvival = a[np.logical_not(np.isnan(a))]
    return PasientId,OverAllSurvival


PasientID,OverAllSurvival=ReadInFiles(Survival)


# Function to create Readable pasient numbers from PasientID

def Patien_List():
    Patient_List=[]
    for i in PasientID:
        Standard = 'P'
        OneZero = '0'
        TwoZer0 = '00'
        a = int(i)
        digits = int(math.log10(a))+1 # Use this to code number of digits of 'i'
        if digits == 1:
            PatientNum = Standard + TwoZer0 + str(a)
        elif digits == 2:
            PatientNum = Standard + OneZero + str(a)
        else:
            PatientNum = Standard + str(a)
        Patient_List.append(PatientNum)
    return Patient_List


# Create a function that can take in different groups of Pasient.
# Compare it to what is available in file
# Extract Outcome

def OutComeOfPatientList(PatientGroup):
    Utfall = []
    Pasienter = []
    Patient_List = Patien_List()
    for j in PatientGroup:
        for i,k in enumerate (Patient_List):
            if k == j:
                Pasienter.append(k)
                U = OverAllSurvival[i]
                Utfall.append(U)
    return Utfall,Pasienter

PasientID,OverAllSurvival=ReadInFiles(Ecog)


# ECOG, og pasient nummer for 3 pasient grupper
ECOG_ALL,PasietNumbAll = OutComeOfPatientList(AllPasientNumbers)
ECOG_HPVPOS,PasietNumbPOS = OutComeOfPatientList(PatientNumbeRHPVOne)
ECOG_HPVNEG,PasietNumbNEG = OutComeOfPatientList(PatientNumbeRHPVZero)

PasientID,OverAllSurvival=ReadInFiles(MTV)


#MTV-U fra filer.
MTV_ALL_O,PasietNumbAll = OutComeOfPatientList(AllPasientNumbers)
MTV_HPVPOS_O,PasietNumbPOS = OutComeOfPatientList(PatientNumbeRHPVOne)
MTV_HPVNEG_O,PasietNumbNEG = OutComeOfPatientList(PatientNumbeRHPVZero)

MTV_FILE = []
PatientNum=[]
PasietNumbNEG_File = []
for j,i in enumerate (MTV_HPVNEG_O):
    if np.isnan(i):
        pass
    else:
        MTV_FILE.append(i)
        PatientNum.append(PatientNumbeRHPVZero[j])

PasientID,OverAllSurvival=ReadInFiles(Survival)
Outcome_file,PasietNumbNEG_file = OutComeOfPatientList(PatientNum) 
#SUV_HPVNEG_file = SUV_samler(PasietNumbNEG_file)
print(Outcome_file)


PasientID,OverAllSurvival=ReadInFiles(Survival)
# Utfall, og pasient nummer for 3 pasient grupper
Outcome_ALL,PasietNumbAll = OutComeOfPatientList(AllPasientNumbers)
Outcome_HPVPOS,PasietNumbPOS = OutComeOfPatientList(PatientNumbeRHPVOne)
Outcome_HPVNEG,PasietNumbNEG = OutComeOfPatientList(PatientNumbeRHPVZero)
'''# Extract SUV for 3 pasient groups
SUV_ALL = SUV_samler(PasietNumbAll)
SUV_HPVPOS = SUV_samler(PasietNumbPOS)
SUV_HPVNEG = SUV_samler(PasietNumbNEG)'''

%store  SUV_ALL
%store  SUV_HPVPOS
%store  SUV_HPVNEG
%store Outcome_HPVNEG
%store Outcome_ALL
%store Outcome_HPVPOS
%store SUV_HPVNEG_file



In [ ]:
#Calculate Tumor Volume for 3 pasient groups
TumorVolume_ALL = TumorVolumeFunction(SUV_ALL)
TumorVolume_POS = TumorVolumeFunction(SUV_HPVPOS)
TumorVolume_NEG = TumorVolumeFunction(SUV_HPVNEG)

# Calculate Max SUV for 3 pasient groups
SUVMAX_ALL = SUVMAXMINE(SUV_ALL)
SUVMAX_POS = SUVMAXMINE(SUV_HPVPOS)
SUVMAX_NEG = SUVMAXMINE(SUV_HPVNEG)

#Calculate MTV FOR 3 PasientGroups
MTV_ALL = MTV(SUV_ALL)
MTV_POS = MTV(SUV_HPVPOS)
MTV_NEG = MTV(SUV_HPVNEG)

#Calculate TLG FOR 3 PasientGroups
TLG_ALL = TLG(SUV_ALL)
TLG_POS = TLG(SUV_HPVPOS)
TLG_NEG = TLG(SUV_HPVNEG)

# Ecog
ECOG_ALL = ECOG_ALL
ECOG_POS = ECOG_HPVPOS
ECOG_NEG = ECOG_HPVNEG

# MTV file
MTV_NEG_file = MTV(SUV_HPVNEG_file)

### GTV and MTV predictors are saved for 3 pasient groups

In [ ]:
# Tumor Volume predictors for 3 pasient groups
X_ALL_GTV = np.array(np.ravel(TumorVolume_ALL).reshape(-1,1))
X_HPVPOS_GTV = np.array(np.ravel(TumorVolume_POS).reshape(-1,1))
X_HPVNEG_GTV = np.array(np.ravel(TumorVolume_NEG).reshape(-1,1))
# MTV predictors for 3 pasient groups
X_ALL_MTV = np.array(np.ravel(MTV_ALL).reshape(-1,1))
X_HPVPOS_MTV = np.array(np.ravel(MTV_POS).reshape(-1,1))
X_HPVNEG_MTV = np.array(np.ravel(MTV_NEG).reshape(-1,1))
# TLG predictors for 3 pasient groups
X_ALL_TLG = np.array(np.ravel(TLG_ALL).reshape(-1,1))
X_HPVPOS_TLG = np.array(np.ravel(TLG_POS).reshape(-1,1))
X_HPVNEG_TLG = np.array(np.ravel(TLG_NEG).reshape(-1,1))
# SUVMAX predictors for 3 pasient groups
X_ALL_SUVMAX = np.array(np.ravel(SUVMAX_ALL).reshape(-1,1))
X_HPVPOS_SUVMAX = np.array(np.ravel(SUVMAX_POS).reshape(-1,1))
X_HPVNEG_SUVMAX = np.array(np.ravel(SUVMAX_NEG).reshape(-1,1))
# Ecog predictors for 3 pasient groups
X_ALL_ECOG = np.array(np.ravel(ECOG_ALL).reshape(-1,1))
X_HPVPOS_ECOG = np.array(np.ravel(ECOG_POS).reshape(-1,1))
X_HPVNEG_ECOG = np.array(np.ravel(ECOG_NEG).reshape(-1,1))

# MTV file
X_HPVNEG_MTV_file = np.array(np.ravel(MTV_NEG_file).reshape(-1,1))




# Create a profile

In [ ]:
# Function that extracts patient coordinates
# Here we run through an array for x,y and z
# I choose those that fullfil x**2+y**2+z**2 = r**2.
def CircleCoordinates(radiusLength):
    # Create arrays for x,y and z
    y = np.linspace(-radiusLength,radiusLength,2*radiusLength+1)
    z = np.linspace(-radiusLength,radiusLength,2*radiusLength+1)
    x = np.linspace(-radiusLength,radiusLength,2*radiusLength+1)
    # Loop over and get possible whole coordinates from circle arrays x,y and z
    o=[] # Collects x,y and z that fullfil r**2=x**2+y**2+z**2
    P=[] # Collcects the coordinates, o.
    for i in range(radiusLength):
        for j in y:
            for k in x:
                for p in z:
                    if j**2+k**2+p**2 == i**2:
                        o.append(j)
                        o.append(k)
                        o.append(p)
        P.append(o)
        o=[]
    # Group the different coordinates for each radius in 3 and 3.
    Final=[]
    for i in P:
        N = 3
        subList = [i[n:n+N] for n in range(0, len(i), N)]
        Final.append(subList)
    return Final
# Make one function that extracts information about Patient. This cuts down computation time
def CenterXYZ(PatientNumber):
    Dummy,X,Y,Z = create_dummy_matrix(PatientNumber)
    Center_x =(np.amax(X)+np.amin(X))/2
    Center_y =(np.amax(Y)+np.amin(Y))/2
    Center_z =(np.amax(Z)+np.amin(Z))/2
    return Center_x,Center_y,Center_z,X,Y,Z


# Create a function that adds center of tumor
def Radius_Array(Center_x,Center_y,Center_z,Array):
    Ko = []
    for i,j in enumerate(Array):
        x = j[0]+int(Center_x)-1
        y = j[1]+int(Center_y)-1
        z = j[2]+int(Center_z)-1
        Ko.append(x)
        Ko.append(y)
        Ko.append(z)
    N = 3
    SubList = [Ko[n:n+N] for n in range(0, len(Ko), N)]

    return SubList

# This function finds out WHERE the sphere and the tumor actualy do intersect.
# This is done for every radius.
def intersect(array,X,Y,Z):
    index_list = []
    for i, j in enumerate(Z):
        x_values = X[i] # x-coordinate
        y_values = Y[i] # y-coordinate
        for å in array:#
            if å[0] == x_values and å[1] == y_values and å[2] == j:
                    index_list.append(i)
    return index_list


# This function runs over all sphere radiuses and adds tumor center using function radius_array
# Then it runs over all radius, and finds the intersection point each radius has with tumor.
# This is stored in array index_radiuses.
def RadiusTumorCenter(RadiusLength,PatientNumber):
    Center_x,Center_y,Center_z,X,Y,Z = CenterXYZ(PatientNumber)
    Sphere_Coordinates = CircleCoordinates(RadiusLength)
    AddedTumorCenter=[]
    for i in Sphere_Coordinates:
        a = Radius_Array(Center_x,Center_y,Center_z,i)
        AddedTumorCenter.append(a)
    Index_Radiuses=[]
    for i in AddedTumorCenter:
        Index_Radius =  intersect(i,X,Y,Z)
        Index_Radiuses.append(Index_Radius)
    return Index_Radiuses

In [ ]:
# Function that calculates mean SUV for EACH radius.
def Pet_mean_for_each_radius(Radius_Arrayy,Pet_Values):
    PET_values_intersections = []
    for i in Radius_Arrayy:
        u = Pet_Values[i]
        PET_values_intersections.append(u)
    MEANPET = np.mean(PET_values_intersections)
    return MEANPET

# Takes in the indexes given by RadiusTumorCenter function
# Brings out the PET values with those indexes, and calculate mean PET for every sphere going out.
def Read_Extract_PET(PatientNumber,Length_Of_Radius):
    # Data is read in
    Part_readfile = '/volumes/LaCie/Sagal/HN_ANON_09102017/Patient_Folders/'
    strek ='/'
    file_text = '_info.txt'
    file_raw = '_raw_data.txt'
    InfoTextFile = Part_readfile + PatientNumber + strek + PatientNumber + file_text
    RawTextFile  = Part_readfile + PatientNumber + strek + PatientNumber + file_raw
    Index_data = Func_Concat_Header_Array_Index_PT_CT(RawTextFile,Index_type='PT')
    PET_VALUES = []
    # Do not remeber. But it is important
    for i in Index_data:
        for j in i:
            if j == 'Hi':
                pass
            else:
                PET_VALUES.append(j)
    # To chose correct array number FROM PET_VALUES, ARRAY NUM IS USED.
    ArrayName,ArrayNum,PatientWithTumor = CorrectTumorChoice(PatientNumber)
    # Need to define a so that we chose the correct array
    a = 2*ArrayNum +1
    Pet_Values0 = PET_VALUES[a]
    StoredPET = []
    Radius_Index_Array=RadiusTumorCenter(Length_Of_Radius,PatientNumber,)

    for i in Radius_Index_Array:
        b = Pet_mean_for_each_radius(i,Pet_Values0)
        if b <1000: # Do this to exclude NaN
            c = b
        else:
            c = 0
        StoredPET.append(c)
    return StoredPET

## Profile plot, and extraction of the MEAN SUV INTERCEPTION

In [ ]:
# From function read_extract_pet there are zeros due to radius length included
# This function we use to remove those.

def remove_zeros_end(PatientNumber,ArrayLength):
    PET_SUV = Read_Extract_PET(PatientNumber,ArrayLength)
    PETSUV=[]
    for i in PET_SUV[10:]:
        if i == 0:
            pass
        else:
            PETSUV.append(i)
    a = PET_SUV[:10]
    for i in PETSUV:
        a.append(i)

    return a  

def plot_Patients_HPV_NEG(PatientNumber,PET_SUV):
    # Y axis values
    y = PET_SUV
    # corresponding X axis values
    x = np.linspace(0,len(PET_SUV)/(len(PET_SUV)),len(PET_SUV))

    # plotting the points
    plt.plot(x, y,'*')

    # naming the x axis
    plt.xlabel('Radius in pixel')
    # naming the y axis
    plt.ylabel('SUV')

    # giving a title to my graph
    plt.title(PatientNumber)

    # function to show the plot
    plt.show()
    
    return PET_SUV


# Function below extracts PET_SUV_ALL, but we have stored it. So we dont need to run it. 
'''def plot_Patients_HPV_NEG(PatientNumber,ArrayLength):
    PET_SUV = remove_zeros_end(PatientNumber,ArrayLength)
    # x axis values
    y = PET_SUV
    # corresponding y axis values
    x = np.linspace(0,len(PET_SUV)/(len(PET_SUV)),len(PET_SUV))


    # plotting the points
    plt.plot(x, y,'*')

    # naming the x axis
    plt.xlabel('Radius in pixel')
    # naming the y axis
    plt.ylabel('SUV')

    # giving a title to my graph
    plt.title(PatientNumber)

    # function to show the plot
    plt.show()
    
    return PET_SUV'''

'''PET_SUV_ALL=[]
for j,i in enumerate (PasietNumbNEG):
    print('Outcome',Outcome_HPVNEG[j])
    PET_SUV = plot_Patients_HPV_NEG(i,50)
    PET_SUV_ALL.append(PET_SUV)
# FUNCTION OVER IS USED TO STORE pet_suv'''

In [ ]:
%store -r PET_SUV_ALL
for j,i in enumerate(PET_SUV_ALL):
    print('Outcome',Outcome_HPVNEG[j])
    PatientNumber = PasietNumbNEG[j]
    PLOT_DUMMY(PatientNumber)
    PET_SUV = plot_Patients_HPV_NEG(PatientNumber,i)
    print('Interpolated PET_SUV')
    PET_SUV_INTERP= plot_Patients_HPV_NEG(PatientNumber,PET_SUV_ALL_INTERP[j])



## Interpolation of mean suv interception

In [ ]:
# Run trough and interpolate PET_SUV_
PET_SUV_ALL_INTERP=[]
Radius = np.linspace(0,50,50)
base_radius = np.linspace(0, 1, 101)
for i in PET_SUV_ALL:
    a = len(i)
    Radius = np.linspace(0,1,a)
    SUV_CENTER_INTERP = np.interp(base_radius, Radius, i)
    PET_SUV_ALL_INTERP.append(SUV_CENTER_INTERP)

# Here, all sphere Radius interpolated are stored.
# I run through PET_SUV_ALL_INTERP, and only extract first radius value
# Add it to a list, and continue until I have Sphere_Radius_all which contains SUV for each radius length for all patients. 
Sphere_Radius_all=[]
for k in range(len(base_radius)):
    Sphere_Radius=[]
    for i,j in enumerate(PET_SUV_ALL_INTERP):
        a = j[k]
        Sphere_Radius.append(a)
    Sphere_Radius_all.append(Sphere_Radius)
    Sphere_Radius=[]

# Here I prepare so that analysis can be performed. 
Sphere_Radius_all_processed = []
for i in Sphere_Radius_all:
    SphereRadius = np.array(np.ravel(i).reshape(-1,1))
    processed_SphereRadius = np.concatenate([SphereRadius],axis=1)
    Sphere_Radius_all_processed.append(processed_SphereRadius)


from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline

# Below gridsearch we use it to calculate Accuracy together with F1 score using a Kfold = 10

'''def GridSearch(scoring,Output,split,Predictor):
    #define cross-validation method to use
    #cv = LeaveOneOut()
    #define cross-validation method to use
    cv = KFold(n_splits=split, random_state=1, shuffle=True)
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression()
    #logreg = SVC()
    # fit the model with data
    ypart = Output
    xpart = Predictor
    logreg.fit(xpart, ypart)
    parameters = { 
    'C'       : np.logspace(-10,10,1000)}
    ##parameters = { 
    #'C'       #: (C)}
    clf = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = scoring,
                           cv = cv)
    clf.fit(xpart, ypart)
    print("Tuned Hyperparameters :", clf.best_params_)
    print("Accuracy:",clf.best_score_)
    return clf.best_score_'''

'''Accuracy=[]
for i,j in enumerate(Sphere_Radius_all_processed):
    print(i)
    array =(GridSearch('accuracy',Outcome_HPVNEG,70,j))
    Accuracy.append(array)

F1_score=[]
for i,j in enumerate(Sphere_Radius_all_processed):
    print(i)
    Array=(GridSearch('f1',Outcome_HPVNEG,10,j))
    F1_score.append(Array)'''

'''%store Accuracy
%store F1_score'''


'''plt.figure()
#plt.plot(base_radius,F1_score)
plt.figure(base_radius,Accuracy)
plt.show()'''

In [ ]:
'''# In this code, I remove those patients that include 
ko = []
Outcome = []
for j,i in enumerate (PET_SUV_ALL_INTERP):
    if i[0] == 0.0 or i[1]==0.0 or i[2]==0.0 or i[3]==0.0:
        pass
    else:
        ko.append(i)
        Outcome.append(Outcome_HPVNEG[j])
        

Sphere_Radius_all=[]
for k in range(len(base_radius)):
    Sphere_Radius=[]
    for i,j in enumerate(ko):
        a = j[k]
        Sphere_Radius.append(a)
    Sphere_Radius_all.append(Sphere_Radius)
    Sphere_Radius=[]

# Here I prepare so that analysis can be performed. 
Sphere_Radius_all_processed = []
for i in Sphere_Radius_all:
    SphereRadius = np.array(np.ravel(i).reshape(-1,1))
    processed_SphereRadius = np.concatenate([SphereRadius],axis=1)
    Sphere_Radius_all_processed.append(processed_SphereRadius)'''

        

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
def gridSearch(scoring,Output,Predictor):
    #define cross-validation method to use
    #cv = LeaveOneOut()
    #define cross-validation method to use
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression()
    #logreg = SVC()
    # fit the model with data
    ypart = Output
    xpart = Predictor
    logreg.fit(xpart, ypart)
    parameters = { 
    'C'       : np.logspace(-10,10,1000)}
    ##parameters = { 
    #'C'       #: (C)}
    clf = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = scoring)
    clf.fit(xpart, ypart)
    print("Tuned Hyperparameters :", clf.best_params_)
    print("Accuracy:",clf.best_score_)
    return clf.best_score_


'''auc_score=[]
for i,j in enumerate(Sphere_Radius_all_processed):
    print(i)
    Array=(gridSearch('roc_auc',Outcome,j))
    auc_score.append(Array)'''

f1_score=[]
for i,j in enumerate(Sphere_Radius_all_processed):
    print(i)
    Array=(gridSearch('f1',Outcome,j))
    f1_score.append(Array)

In [ ]:
'''%store -r Accuracy
%store -r F1_score'''
%store f1_score
%store auc_score

In [ ]:
# Use below code to extract index of which interpolated PET_SUV_SPHERE gives highest score
Highest_Accuracy = 0
for j,i in enumerate (Accuracy):
    if Highest_Accuracy < i:
        Highest_Accuracy = i
        Index = j
# Optimal Sphere is stored here
Best_Sphere_Radius_all_processed = Sphere_Radius_all_processed[Index]

In [ ]:

plt.figure()
plt.plot(base_radius,f1_score,label='F1_SCORE')
plt.plot(base_radius,auc_score,label='auc_score')
plt.plot(base_radius,Accuracy,label='Accuracy')
plt.xlabel('Radius')
plt.ylabel('Score')
plt.title('F1 score vs Accuracy')
plt.legend()
plt.show()



## Split of data. Also deciding on features.

In [ ]:
from sklearn.model_selection import train_test_split
# GTV DATA
processed_Data_All_GTV = np.concatenate([X_ALL_GTV],axis=1)
processed_Data_HPVPOS_GTV = np.concatenate([X_HPVPOS_GTV],axis=1)
processed_Data_HPVNEG_GTV = np.concatenate([X_HPVNEG_GTV],axis=1)

#SUV MAX DATA
processed_Data_All_SUVMAX = np.concatenate([X_ALL_SUVMAX],axis=1)
processed_Data_HPVPOS_SUVMAX = np.concatenate([X_HPVPOS_SUVMAX],axis=1)
processed_Data_HPVNEG_SUVMAX = np.concatenate([X_HPVNEG_SUVMAX],axis=1)

# MTV DATA
processed_Data_All_MTV = np.concatenate([X_ALL_MTV],axis=1)
processed_Data_HPVPOS_MTV = np.concatenate([X_HPVPOS_MTV],axis=1)
processed_Data_HPVNEG_MTV = np.concatenate([X_HPVNEG_MTV],axis=1)

# TLG DATA
processed_Data_All_TLG = np.concatenate([X_ALL_TLG],axis=1)
processed_Data_HPVPOS_TLG = np.concatenate([X_HPVPOS_TLG],axis=1)
processed_Data_HPVNEG_TLG = np.concatenate([X_HPVNEG_TLG],axis=1)

# ECOG DATA
processed_Data_All_ECOG = np.concatenate([X_ALL_ECOG],axis=1)
processed_Data_HPVPOS_ECOG = np.concatenate([X_HPVPOS_ECOG],axis=1)
processed_Data_HPVNEG_ECOG = np.concatenate([X_HPVNEG_ECOG],axis=1)

# MTV file
processed_Data_HPVNEG_MTV_file = np.concatenate([X_HPVNEG_MTV_file],axis=1)




# Best Best_Sphere_Radius_all_processed

#Sphere_Radius = Sphere_Radius_all_processed[Index]


def AllData(Input,Predictor):
    if Input == 'NEG' and Predictor == 'GTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart = (processed_Data_HPVNEG_GTV)
    elif Input == 'NEG' and Predictor == 'MTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1)) 
        xpart = (processed_Data_HPVNEG_MTV)
    if Input == 'NEG' and Predictor == 'SUVMAX':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart = (processed_Data_HPVNEG_SUVMAX)
    if Input == 'NEG' and Predictor == 'TLG':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart = (processed_Data_HPVNEG_TLG)
    if Input == 'NEG' and Predictor == 'ECOG':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart = (processed_Data_HPVNEG_ECOG)
    if Input == 'NEG_file' and Predictor == 'MTV':
        ypart = (np.ravel(Outcome_file).reshape(-1,1))
        xpart = (processed_Data_HPVNEG_MTV_file)
    elif Input == 'POS' and Predictor == 'GTV' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))      
        xpart = (processed_Data_HPVPOS_GTV)
    elif Input == 'POS' and Predictor == 'MTV' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))        
        xpart = (processed_Data_HPVPOS_MTV)
    elif Input == 'POS' and Predictor == 'SUVMAX' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))        
        xpart = (processed_Data_HPVPOS_SUVMAX)
    elif Input == 'POS' and Predictor == 'TLG' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))        
        xpart = (processed_Data_HPVPOS_TLG)
    elif Input == 'POS' and Predictor == 'ECOG' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))        
        xpart = (processed_Data_HPVPOS_ECOG)
    elif Input == 'All' and Predictor == 'GTV' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart =(processed_Data_All_GTV)
    elif Input == 'All' and Predictor == 'MTV' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart=(processed_Data_All_MTV)
    elif Input == 'All' and Predictor == 'SUVMAX' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart = (processed_Data_All_SUVMAX)
    elif Input == 'All' and Predictor == 'TLG' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart = (processed_Data_All_TLG)
    elif Input == 'All' and Predictor == 'ECOG' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart = (processed_Data_All_ECOG)
    elif Input == 'All' and Predictor == 'SUVMAX+MTV' :
        ypart = (np.ravel(Outcome_ALL).reshape(-1,1))
        xpart1 = (processed_Data_All_SUVMAX)
        xpart2 = (processed_Data_All_MTV)
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'SUVMAX+MTV' :
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_SUVMAX)
        xpart2 = (processed_Data_HPVNEG_MTV)
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'SUVMAX+GTV' :
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_SUVMAX)
        xpart2 = (processed_Data_HPVNEG_GTV)
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'SUVSphere+MTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_MTV)
        xpart2 = Sphere_Radius
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'ECOG+GTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_GTV)
        xpart2 = processed_Data_HPVNEG_ECOG
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'SUVCENTERinterp+MTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_MTV)
        xpart2 = (processed_Data_HPVNEG_SUVCENTER_interp)
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'SUVCENTERinterp+GTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_GTV)
        xpart2 = processed_Data_HPVNEG_SUVCENTER_interp
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'NEG' and Predictor == 'ECOG+MTV':
        ypart = (np.ravel(Outcome_HPVNEG).reshape(-1,1))
        xpart1 = (processed_Data_HPVNEG_GTV)
        xpart2 = processed_Data_HPVNEG_ECOG
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    elif Input == 'POS' and Predictor == 'SUVMAX+MTV' :
        ypart = (np.ravel(Outcome_HPVPOS).reshape(-1,1))
        xpart1 = (processed_Data_HPVPOS_SUVMAX)
        xpart2 = (processed_Data_HPVPOS_MTV)
        xpart = np.concatenate((xpart1, xpart2), axis=1)
    return ypart,xpart

## Oversikt over Data
- Kan det være noe for HPV negative pasienter at vi kaster ut pasient med tumor = 150?

In [ ]:
# Plot of Logistic Function for our data. As can be seen from plots below, 
# we cant see any "mønster." I will remove the smallest tumors. All those that have
# Add your plots here, to see out. £

print('HPV NEGATIVE PATIENTS')
print('GTV')
plt.figure(1)
Ypart,Xpart = AllData('NEG','GTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('GTV')
plt.ylabel('Outcome')
plt.title('GTV vs Outcome')
plt.show()

print('MTV')
plt.figure(1)
Ypart,Xpart = AllData('NEG','MTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('MTV')
plt.ylabel('Outcome')
plt.title('MTV vs Outcome')
plt.show()


print('HPV POSITIVE PATIENTS')
print('GTV')
plt.figure(1)
Ypart,Xpart = AllData('POS','GTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('GTV')
plt.ylabel('Outcome')
plt.title('GTV vs Outcome')
plt.show()

print('MTV')
plt.figure(1)
Ypart,Xpart = AllData('POS','MTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('MTV')
plt.ylabel('Outcome')
plt.title('MTV vs Outcome')
plt.show()

print('ALL PATIENTS')
print('GTV')
plt.figure(1)
Ypart,Xpart = AllData('All','GTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('GTV')
plt.ylabel('Outcome')
plt.title('GTV vs Outcome')
plt.show()

print('MTV')
plt.figure(1)
Ypart,Xpart = AllData('All','MTV')
plt.plot(Xpart,Ypart,'*')
plt.xlabel('MTV')
plt.ylabel('Outcome')
plt.title('MTV vs Outcome')
plt.show()


## Correlation between GTV and MTV for each patient group together with a plot
- Kan se ut som korrelasjonen er størst ved de minste tumorene. Det er kanskje de største tumorene som har minst korrelasjon.

In [ ]:
r = np.corrcoef(X_ALL_GTV.T, X_ALL_MTV.T)
print('Correlation between GTV and MTV for whole tumor',r)
#Plot
plt.plot(X_ALL_GTV,X_ALL_MTV,'*')
plt.xlabel('GTV')
plt.ylabel('MTV')
plt.show()

r = np.corrcoef(X_HPVPOS_GTV.T, X_HPVPOS_MTV.T)
print('Correlation between GTV and MTV for HPV POSITIVE',r)
#Plot
plt.figure(1)
plt.plot(X_HPVPOS_GTV,X_HPVPOS_MTV,'*')
plt.xlabel('GTV')
plt.ylabel('MTV')
plt.ylabel('GTV')
plt.show()

r = np.corrcoef(X_HPVNEG_GTV.T, X_HPVNEG_MTV.T)
print('Correlation between GTV and MTV for HPV NEGATIVE',r)
#Plot
plt.plot(X_HPVNEG_GTV,X_HPVNEG_MTV,'*')
plt.xlabel('GTV')
plt.ylabel('MTV')
plt.show()


r = np.corrcoef(Best_Sphere_Radius_all_processed.T, X_HPVNEG_MTV.T)
print('Correlation between Best_Sphere_Radius_all and MTV for HPV NEGATIVE',r)
#Plot
plt.plot(Best_Sphere_Radius_all_processed,X_HPVNEG_MTV,'*')
plt.xlabel('Best_Sphere_Radius_all')
plt.ylabel('MTV')
plt.show()

## BOX PLOT
- Seems like outliers are more reduced for HPV Negative patients.
- Also outliers reduce more for MTV compared to GTV
- Change this to Violine plot

In [ ]:
## Box plot
# Creating dataset
np.random.seed(10)
 
data_1 = X_ALL_GTV
data_2 = X_ALL_MTV
data_3 = X_HPVPOS_GTV
data_6 = X_HPVPOS_MTV
data_4 = X_HPVNEG_GTV
data_5 = X_HPVNEG_MTV
data = [data_1, data_2, data_3,data_6, data_4,data_5]
 
fig = plt.figure(figsize =(10, 7))
 
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])

ax.set_title('axes title')
 
# Creating plot
bp = ax.boxplot(data)

plt.ylabel('Tumor Volume')
 
# show plot
plt.show()

## Logit function for all three patient groups.

## GRID SEARCH
- Noe jeg syntes er rart her er XTrain, og hele data gir samme resultat. Undersøk dette nærmere om det stemmer. Vi har fått bedre presisjon for HPV POS, og alle pasienter. Men, HPV negative er uendret. Se nærmere. 
- Se om du kan få bedre resultater om du inkluderer andre parametere enn C inverse regulirization

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

def GridSearch(Input,Predictor):
    #define cross-validation method to use
    #cv = LeaveOneOut()
    #define cross-validation method to use
    cv = KFold(n_splits=65, random_state=1, shuffle=True)
    # instantiate the model (using the default parameters)
    logreg = LogisticRegression()
    #logreg = SVC()
    # fit the model with data
    ypart,xpart = AllData(Input,Predictor)
    logreg.fit(xpart, ypart)
    parameters = { 
    'C'       : np.logspace(-5,5,1000)}
    #parameters = { 
    #'C'       : (C)}
    clf = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = cv)
    clf.fit(xpart, ypart)
    print("Tuned Hyperparameters :", clf.best_params_)
    print("Accuracy:",clf.best_score_)
    return clf.best_score_



# Lage Funksjon som iterer over forskjellige C-verdier.
# Grunnet tid, skal jeg se på følgende:- MTV+suvmax,MTV+SUVCENTER,GTV+SUVCENTER
# Function that searches through and CALCULATES AUC.
def Score_Grid_Search(c,Outcome,Predictor):
    scores=[]
    C =[]
    for k in c:
        Ydata,Xdata = AllData(Outcome,Predictor)
        X = np.array(Xdata) # Predictors stored here
        y = np.array(Ydata)# Outcome
        logreg = LogisticRegression(C = k ).fit(X, y)
        #define cross-validation method to use
        cv = KFold(n_splits=20, shuffle=True)
        #use k-fold CV to evaluate model
        score = cross_val_score(logreg, X, y, scoring='accuracy',
                                 cv=cv, n_jobs=-1)
        scores.append(np.mean(score))
        C.append(c)
    return scores,C


def Extract_BEST_Score(c,Outcome,Predictor):
    scores,C=Score_Grid_Search(c,Outcome,Predictor)
    Best_Score=0
    for i,j in enumerate (scores):
        if j >Best_Score:
            Best_Score=j
            Best_C = C[i]
    return Best_Score,Best_C

# Function that searches through and CALCULATES AUC.
def Auc_Grid_Search(c,Outcome,Predictor):   
    AUCS=[]
    A=[]
    tprs =[]
    for k in c:
        Ydata,Xdata = AllData(Outcome,Predictor)
        X = np.array(Xdata) # Predictors stored here
        y = np.array(Ydata)# Outcome
        logreg = LogisticRegression(C = k ).fit(X, y)
        y_score = logreg.predict_proba(X)
        fpr, tpr, _ = roc_curve(y, y_score[:, 1])
        AUC = auc(fpr, tpr)
        AUCS.append(AUC)
        print(AUC)
    return AUCS,c

# Function extracting high AUC with corresponding C. 
def Extract_C_AUC(c,Outcome,Predictor):
    AUCS,c=Auc_Grid_Search(c,Outcome,Predictor)
    p = 0;
    B = 0;
    for j,i in enumerate(AUCS):
        if i > B:
            B = i           
    h = []
    p=[]
    for i,j  in enumerate (AUCS):
        if j >=B:
            h.append(j)
            o = float(c[i])
            p.append(o)
    return h,p





'''print('Results For All Patients GTV',GridSearch('All','GTV'))
print('Results For All Patients MTV',GridSearch('All','MTV'))
print('Results For All Patients SUVMAX',GridSearch('All','SUVMAX'))
print('Results For HPV NEG GTV',GridSearch('NEG','GTV'))
print('Results For HPV NEG MTV',GridSearch('NEG','MTV'))
print('Results For hpv NEG  SUVMAX',GridSearch('NEG','SUVMAX'))
print('Results For HPV POS GTV',GridSearch('POS','GTV'))
print('Results For HPV POS MTV',GridSearch('POS','MTV'))
print('Results For HPV POS SUVMAX',GridSearch('POS','SUVMAX'))
print('Results For HPV NEG MTV and SuvMax',GridSearch('HPVNEG_Double','SUVMAX+MTV'))
print('Results For HPV POS MTV and SuvMax',GridSearch('HPVPOS_Double','SUVMAX+MTV'))
print('Results For ALL Patients MTV and SUVMax',GridSearch('All_Double','SUVMAX+MTV'))'''

In [ ]:
print(GridSearch('NEG_file','MTV'))


In [ ]:
from sklearn import metrics
Ydata,Xdata = AllData('NEG_file','MTV')
X = np.array(Xdata) # Predictors stored here
y = np.array(Ydata)# Outcome
logreg = LogisticRegression(C = 40 ).fit(X, y)
y_score = logreg.predict_proba(X)
fpr, tpr, _ = metrics.roc_curve(y, y_score[:, 1])
AUC = metrics.auc(fpr, tpr)
print(AUC)

In [ ]:
- Recall mot Presisjon
- F1 score.
- Interpolering, fjerner pasienter med spesiell tumor. 

## All patients GridSearch HPV NEG PATIENTS

In [ ]:
# Chose what values regularization parameter should span
c = np.logspace(-10,10,1000)

# Calculation of accuracy while using the BEST AUCS from function Extract_C_AUC()
AUCS_GTV,C_GTV = Auc_Grid_Search(c,'NEG','MTV')
#a,b=(GridSearch('NEG','GTV',C_GTV))
print('Best GTV SCORE',(AUCS_GTV),'BEST C',len(C_GTV))
'''AUCS_MTV,C_MTV = Extract_C_AUC(c,'NEG','GTV')
cc,d=(Extract_BEST_Score(C_MTV,'NEG','MTV'))
print('Best MTV SCORE',cc,'BEST C',d)
AUCS_MTVSUVMAX,C_MTVSUVMAX = Extract_C_AUC(c,'HPVNEG_Double','SUVMAX+MTV')
aa,bb=(Extract_BEST_Score(C_MTVSUVMAX,'HPVNEG_Double','SUVMAX+MTV'))
print('Best MTV+SUVMAX SCORE',aa,'BEST C',bb)
AUCS_GTVSUVMAX,C_GTVSUVMAX = Extract_C_AUC(c,'HPVNEG_Double','SUVMAX+GTV')
ff,dd=(Extract_BEST_Score(C_GTVSUVMAX,'HPVNEG_Double','SUVMAX+GTV'))
print('Best GTV+SUVMAX SCORE',ff,'BEST C',dd)
AUCS_GTVSUVCENTER,C_GTVSUVCENTER = Extract_C_AUC(c,'HPVNEG_Double','SUVCENTER+GTV')
ee,gg=(Extract_BEST_Score(C_GTVSUVCENTER,'HPVNEG_Double','SUVCENTER+GTV'))
print('Best GTV+SUVCENTER SCORE',ee,'BEST C',gg)
AUCS_GTVSUVMAX,C_MTVSUVMAX = Extract_C_AUC(c,'HPVNEG_Double','SUVCENTER+MTV')
hh,ii=(Extract_BEST_Score(C_MTVSUVMAX,'HPVNEG_Double','SUVCENTER+MTV'))
print('Best MTV+SUVMAX SCORE',hh,'BEST C',ii)'''

# Du kan kjøre på nytt, og velge C-verdiene med høyest AUC

### ROC CURVES
The ROC curves are plotted for three patient groups. This is to get a view, and a visualization of how things look.
- STD er sterkt påvirket av mengde data. Vi ser at den store pasient gruppen har mindre std sammenlignet med de andre. Nesten halvparten. De andre gruppe pasientene har like stor std. Kanskje nederland data vil forberede dette?

In [ ]:
from sklearn import metrics
from sklearn.metrics import auc
#from scikitplot.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings('ignore')


# Funcion that produces extrapolated tprs with a base fpr. Also it plots a number of ROV-curves.
def ROC_tpr_fpr(C,Input,Predictor,size):
    Ydata,Xdata = AllData(Input,Predictor)
    X = np.array(Xdata) # Predictors stored here
    y = np.array(Ydata)# Outcome
    logreg = LogisticRegression(C = C ).fit(X, y)
    y_score = logreg.predict_proba(X)
    fpr, tpr, _ = roc_curve(y, y_score[:, 1])
    AUC = auc(fpr, tpr)
    print('AUC for whole dataset',AUC)
    plt.plot(fpr, tpr, 'r', alpha=1)
    base_fpr = np.linspace(0, 1, 101) # I will extrapolate this later tpr. 
    idx = np.arange(0, len(y))
    # Will use this to later shuffle my data
    tprs = []
    aucs = []
    for j in np.random.randint(0, high=10000, size=size): # Velger tilfeldig j mellom 0 og 10 000. Velger ut 10 tall.
        np.random.shuffle(idx) # Jeg mixer tallene i idx
        cv = StratifiedKFold(n_splits=5,random_state=j) # Cross validation. Split my data in 5
        for i, (train, test) in enumerate(cv.split(X,y)):
            logreg = LogisticRegression(C = C ).fit(X[idx][train], y[idx][train])
            y_score = logreg.predict_proba(X[idx][test])
            fpr, tpr, _ = roc_curve(y[idx][test], y_score[:, 1])
            AUC = auc(fpr, tpr)
            aucs.append(AUC)
            plt.plot(fpr, tpr, 'b', alpha=0.05)
            tpr = np.interp(base_fpr, fpr, tpr)
            tpr[0] = 0.0
            tprs.append(tpr)
    return tprs,base_fpr,aucs

# Insert Data information here. 
def ROC_PLOT_CURVE(C,Input,Predictor,size):
    tprs,base_fpr,aucs = ROC_tpr_fpr(C,Input,Predictor,size)
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
    mean_auc = auc(base_fpr, mean_tprs)
    std_auc = np.std(aucs)
    print('Mean AUC is',mean_auc,'With a STD',std_auc)


    plt.plot(base_fpr, mean_tprs, 'b')
    plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)

    plt.plot([0, 1], [0, 1],'g--')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

    plt.show()
'''print('ROC CURVE FOR HPV NEG PATIENTS WITH MTV AS PREDICTOR')
print(ROC_PLOT_CURVE(22.4970052592174,'NEG','MTV',10))

print('ROC CURVE FOR HPV NEG PATIENTS WITH GTV AS PREDICTOR')
print(ROC_PLOT_CURVE( 22.4970052592174,'NEG','GTV',10))

print('ROC CURVE FOR HPV NEG PATIENTS WITH SUVMAX AS PREDICTOR')
print(ROC_PLOT_CURVE(1,'NEG','SUVMAX',10))

print('ROC CURVE FOR HPV NEG PATIENTS WITH SUVMAX and MTV AS PREDICTORS')
print(ROC_PLOT_CURVE(0.0001,'HPVNEG_Double','SUVMAX+MTV',30))



print('ROC CURVE FOR HPV POS PATIENTS WITH MTV AS PREDICTOR')
print(ROC_PLOT_CURVE(2.3974349678010785,'POS','MTV',10))

print('ROC CURVE FOR HPV POS PATIENTS WITH GTV AS PREDICTOR')
print(ROC_PLOT_CURVE(44.21499907374505,'POS','GTV',10))

print('ROC CURVE FOR HPV POS PATIENTS WITH SUVMAX AS PREDICTOR')
print(ROC_PLOT_CURVE(1e-10,'POS','SUVMAX',10))

print('ROC CURVE FOR HPV POS PATIENTS WITH SUVMAX and MTV AS PREDICTORS')
print(ROC_PLOT_CURVE(0.4171124612056533,'HPVPOS_Double','SUVMAX+MTV',0))

print('ROC CURVE ALL PATIENTS WITH MTV AS PREDICTOR')
print(ROC_PLOT_CURVE(1e-10,'All','MTV',10))

print('ROC CURVE FOR ALL PATIENTS WITH GTV AS PREDICTOR')
print(ROC_PLOT_CURVE(1e-10,'All','GTV',20))

print('ROC CURVE FOR ALL PATIENTS WITH SUVMAX AS PREDICTOR')
print(ROC_PLOT_CURVE(1e-10,'All','SUVMAX',20))

print('ROC CURVE FOR ALL PATIENTS WITH SUVMAX and MTV AS PREDICTORS')
print(ROC_PLOT_CURVE(1e-10,'HPVPOS_Double','SUVMAX+MTV',20))'''

## Logistic Regression Analysis

### Evaluating Model

In [ ]:
from sklearn.metrics import confusion_matrix
# Accurancy on Training
print('The training accurancy is', logreg.score(XTrain,yTrain))
print('The test accurancy is', logreg.score(XTest,yTest))

#Classification report
print(classification_report(yTrain,y_pred))

# Confusion matrix plot function

# import required modules
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(cm,classes=None,title='Confusion matrix'):
    if classes is not None:
        sns.heatmap(cm,xticklabels = classes,yticklabels=classes,annot=True,annot_kws={'size':30})
    else:
        sns.heatmap(cm,vmin=0.,vmax=1.)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cm = confusion_matrix(yTrain,y_pred)
cm_norm = cm/cm.sum(axis=1).reshape(-1,1)
plot_confusion_matrix(cm_norm,classes=logreg.classes_,title='Confusion Matrix')

## Checking on how outcome VS feature looks.

## Decision Trees analysis

In [ ]:
# Create a function that does analyze on the min_samples_split, and also maximum number of leaves
# What we end up with is the R2-score and MSE as the parameters are changing each at a time!(not at the same time)

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.svm import SVC
from sklearn import metrics



def DecisionTreeAnalyzor(Input,Predictor,depth,parameter,DataType,MinSampleSplit,MaxNumOfLeafNodes):
    global score
    y,X = AllData(Input,Predictor)
    AllScore = []
    ScorieTestTrain = 0
    MaxieDepth = 0
    MiniSampleSplit = 0
    for i in range(1,depth): # Run over different depth levels
        if parameter == 'MinSampleSplit': # Specify here which parameter we want to the analyze on
            Component = MinSampleSplit
        else:
            Component = MaxNumOfLeafNodes
        for j in Component: # Loop of the parameter
            if parameter == 'MinSampleSplit':
                # Chose the parameters. Here max_depth is iterated over deoth, whereas min_samples_split is changing. See that maximum number of leaves is contant = 100
                model = DecisionTreeClassifier(max_depth = i,min_samples_split = j,max_leaf_nodes = 100,min_weight_fraction_leaf=0.001,random_state=42)
            else:
                #Here the maximum number of leaves changes whereas minimum number of samples before split is constantly = 100
                model = DecisionTreeClassifier(max_depth = i,min_samples_split = 100,max_leaf_nodes = j,min_weight_fraction_leaf=0.001,random_state=42)
            #Fit of model
            model.fit(X,y)
            cv = KFold(n_splits=20, shuffle=True)
            #use k-fold CV to evaluate model
            score = cross_val_score(model, X, y, scoring='accuracy',cv=cv, n_jobs=-1)
'''            y_pred_Test = model.predict(XTest)
            y_pred_Train = model.predict(XTrain)
            Score_Train = metrics.accuracy_score(yTrain, y_pred_Train) 
            Score_Test = metrics.accuracy_score(yTest, y_pred_Test)
            scoresum = Score_Train+Score_Test'''
'''            #Calculate R2-score with help of SKLEARN kfold cross validation
            if DataType == 'Train':
                score = metrics.accuracy_score(yTrain, y_pred_Train)
            else:
                score = metrics.accuracy_score(yTest, y_pred_Test)
            if scoresum > ScorieTestTrain and Score_Test > 0.65:
                ScorieTestTrain = scoresum
                MaxieDepth = i
                MiniSampleSplit = j'''
            # Also here, we calculate the MSE. Differentiate between training or Test data
            '''if parameter == 'MinSampleSplit':
                print("Accurancy Score: {:.2f}".format(score),"Depth: {:.2f}".format(i),"MinSampleSplit: {:.2f}".format(j))
            else:
                print("Accurancy Score: {:.2f}".format(score),"Depth: {:.2f}".format(i),"MaxNumOfLeafNodes: {:.2f}".format(j))'''

    #print('Highest Score is ;',HighestScore,'With a Depth',MaxieDepth, 'and MinSampleSplit',MiniSampleSplit)
    return np.mean(score)
# Input parameters
MinSampleSplit = [2,5,7,10,20,30,50,80,100,200,400,500,600,700,800,1000,1600]
MaxNumOfLeafNodes = [2,10,30,60,100,150,250]
#Be sure to delete the "extra" t, OR add if you want to analyze minimum number of samples.
parameter = ('MinSampleSplitt')
DataType = ('Test') # Insert in datatype so that we are sure we are using train data. Or
j = DecisionTreeAnalyzor(30,'MinSampleSplit','Test',MinSampleSplit,MaxNumOfLeafNodes)
print(j)
'''k = DecisionTreeAnalyzor(30,'MinSampleSplit','Train',MinSampleSplit,MaxNumOfLeafNodes)
print('Highest Score Test',j)
print('Highest Score Train',k)'''

In [ ]:
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
# Train/Test split
y,X = AllData('NEG','ECOG+MTV')
dec_tree = tree.DecisionTreeClassifier()
pipe = Pipeline(steps=[('dec_tree', dec_tree)])
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]
min_samples_split = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
min_weight_fraction_leaf=[0]
min_samples_leaf = [1,2,3,4,5,7,10,12,14,15]
parameters = dict(    dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth,dec_tree__min_samples_split=min_samples_split,dec_tree__min_samples_leaf=min_samples_leaf,dec_tree__min_weight_fraction_leaf = min_weight_fraction_leaf)
Kfold=KFold(n_splits=70, shuffle=True)
clf_GS = GridSearchCV(pipe, parameters,scoring = 'accuracy',cv=Kfold)
clf_GS.fit(X, y)
print(clf_GS.best_score_)
print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print('Best min_samples_split:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_split'])
print('Best min_weight_fraction_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_weight_fraction_leaf'])
print('Best min_samples_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_leaf'])
print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])



In [ ]:
0.7714285714285715 for both MTV and MTV+SUVCENTER
0.71 for Ecog

In [ ]:
from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Train/Test split
y,X = AllData('HPVNEG_Double','SUVCENTER+MTV')
import warnings
warnings.filterwarnings('ignore')

dec_tree = RandomForestClassifier()
pipe = Pipeline(steps=[('dec_tree', dec_tree)])
#model = DecisionTreeClassifier(max_depth = i,min_samples_split = j,max_leaf_nodes = 100,min_weight_fraction_leaf=0.001,random_state=42)
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]
min_samples_split = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
min_weight_fraction_leaf=[0]
min_samples_leaf = [1,2,3,4,5,7,10,12,14,15]
parameters = dict(dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth,dec_tree__min_samples_split=min_samples_split,dec_tree__min_samples_leaf=min_samples_leaf,dec_tree__min_weight_fraction_leaf = min_weight_fraction_leaf)
Kfold=KFold(n_splits=70, shuffle=True)
clf_GS = GridSearchCV(pipe, parameters,scoring = 'accuracy',cv=Kfold)
clf_GS.fit(X, y)
print(clf_GS.best_score_)
print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print('Best min_samples_split:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_split'])
print('Best min_weight_fraction_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_weight_fraction_leaf'])
print('Best min_samples_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_leaf'])
print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline




import warnings
warnings.filterwarnings('ignore')

# Train/Test split
y,X = AllData('HPVNEG_Double','SUVCENTER+MTV')


dec_tree = GradientBoostingClassifier()
pipe = Pipeline(steps=[('dec_tree', dec_tree)])
#model = DecisionTreeClassifier(max_depth = i,min_samples_split = j,max_leaf_nodes = 100,min_weight_fraction_leaf=0.001,random_state=42)
max_depth = [2,4,6,8,10,12]
learning_rate=[0.1,0.2,0.3,0.5]
min_samples_split = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
min_samples_leaf = [1,2,3,4,5,7,10,12,14,15]
#parameters = dict(dec_tree__max_depth=max_depth,dec_tree__learning_rate = learning_rate)
parameters = dict(dec_tree__learning_rate = learning_rate,
                      dec_tree__max_depth=max_depth,dec_tree__min_samples_split=min_samples_split,dec_tree__min_samples_leaf=min_samples_leaf,dec_tree__min_weight_fraction_leaf = min_weight_fraction_leaf)

Kfold=KFold(n_splits=50, shuffle=True)
clf_GS = GridSearchCV(pipe, parameters,scoring = 'accuracy',cv=Kfold)
clf_GS.fit(X, y)
print(clf_GS.best_score_)
print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print('Best min_samples_split:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_split'])
print('Best min_weight_fraction_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_weight_fraction_leaf'])
print('Best min_samples_leaf:', clf_GS.best_estimator_.get_params()['dec_tree__min_samples_leaf'])
print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])

In [ ]:
model = DecisionTreeClassifier(max_depth = 2,min_samples_split = 7,max_leaf_nodes = 100,min_weight_fraction_leaf=0,random_state=42)
model.fit(X,y)
print(model.feature_importances_)



By changing we see that sensitive parameters seem to be the depth and MinSampleSplit. MaxNumOfLeafNodes does not contribute much. So, we shall drop the latter. Vi går for depth = 3, og minSampleSplitt = 80. Det virker mest riktig. 

In [ ]:


# Create Decision Tree classifer object
clf = DecisionTreeClassifier(max_depth=4,min_samples_split=2)

# Train Decision Tree Classifer
clf = clf.fit(XTrain,yTrain)

#Predict the response for test dataset
y_pred = clf.predict(XTest)
print(y_pred)
# Model Accuracy, how often is the classifier correct?
print("Accuracy Decision Trees:",metrics.accuracy_score(yTest, y_pred))


from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=2)

#Train the model using the training sets
knn.fit(XTrain, yTrain)

#Predict the response for test dataset
y_pred = knn.predict(XTest)
print('This',y_pred)

print("Accuracy K-nearest :",metrics.accuracy_score(yTest, y_pred))


# Training a SVM classifier using SVC class
svm = SVC(kernel= 'linear', random_state=1, C=0.1)
svm.fit(XTrain, yTrain)
 
# Mode performance
 
y_pred = svm.predict(XTest)
print('Accuracy SVM: %.3f' % accuracy_score(yTest, y_pred))




mlp = MLPClassifier(hidden_layer_sizes=(4,2), solver='adam', max_iter=500)
mlp.fit(XTrain,yTrain)

predict_train = mlp.predict(XTrain)
predict_test = mlp.predict(XTest)
print("Accuracy NN :",metrics.accuracy_score(yTest, predict_test))




# Neural Network

In [ ]:

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


y,x = AllData('NEG','MTV')



std_slc = StandardScaler()
dec_tree = tree.DecisionTreeClassifier()
n_components = list(range(1,2,1))
criterion = ['gini', 'entropy']
min_weight_fraction_leaf=[0,0.1,0.02,0.5]
max_leaf_nodes = [2,5,8,15,25,35]
pipe = Pipeline(steps=[('std_slc', std_slc),('dec_tree', dec_tree)])
print(pipe)
max_depth = [2,4,6,8]
min_samples_split = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
min_weight_fraction_leaf=[0]
min_samples_leaf = [1,2,3,4,5,7,10,12,14,15]
Kfold=KFold(n_splits=5, shuffle=True)

parameters = dict(pca__n_components=n_components,dec_tree__criterion=criterion,dec_tree__max_depth=max_depth,
                  dec_tree__min_samples_split=min_samples_split,
                  dec_tree__min_samples_leaf=min_samples_leaf,dec_tree__max_leaf_nodes=max_leaf_nodes)
clf = GridSearchCV(pipe, parameters, n_jobs=4,cv=Kfold)
clf.fit(X=x, y=y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

y,x = AllData('NEG','MTV')

dec_tree = RandomForestClassifier()
pipe = Pipeline(steps=[('std_slc', std_slc),('dec_tree', dec_tree)])
parameters = dict(dec_tree__criterion=criterion,dec_tree__max_depth=max_depth,
                  dec_tree__min_samples_split=min_samples_split,dec_tree__min_samples_leaf=min_samples_leaf)
Kfold=KFold(n_splits=20, shuffle=True)
clf = GridSearchCV(pipe, parameters, n_jobs=4,cv = Kfold)
clf.fit(X=x, y=y)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_)

# Important Variables are stored here.

In [ ]:
%store PET_SUV_ALL
%store SUV_ALL
%store SUV_HPVPOS
%store SUV_HPVNEG
%store Accuracy
%store F1_score

In [ ]:
%history -g